# Advanced Computing Concepts with Dynamic Compute
__________________
In previous tutorials we explored the fundementals of Dynamic Compute. In this notebook we will overview of a subset of more complex examples of the API, such as:
* Interacting between Mosaics and ImageStacks
* Grouping ImageStacks
* More advanced batch-style, time series analysis

For a more detailed look at all its classes and their available methods please visit the [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html) Documentation page.

In [ ]:
import descarteslabs as dl
import descarteslabs.dynamic_compute as dc
from descarteslabs.dynamic_compute import Mosaic, ImageStack

In [ ]:
from datetime import date
from shapely.wkt import loads
import matplotlib.pyplot as plt

First, we will set up some global variables and set up our input image stack and mosaic objects. Here we will work with a stack of Sentinel-2 imagery corresponding to the date range of May to August, 2022 and a mosaic of 2022's [Cropland Data Layer](https://app.descarteslabs.com/explorer/datasets/usda:cdl:v1):

In [ ]:
start_date = "2022-05-01"
end_date = "2022-08-01"
pid = "esa:sentinel-2:l2a:v1"
cdl_pid = "usda:cdl:v1"
bands = "nir red green blue"

In [ ]:
if float(dc.__version__[:3]) >= 1.3:
    # this is the >=1.3.0 dynamic-compute version
    s2_stack = ImageStack.from_product_bands(
        pid, bands, start_datetime=start_date, end_datetime=end_date
    ).filter(dl.catalog.properties.cloud_fraction < 0.2)
else:
    # this is the <1.3.0 dynamic-compute version
    s2_stack = ImageStack.from_product_bands(
        pid, bands, start_datetime=start_date, end_datetime=end_date
    ).filter(lambda x: x.cloud_fraction < 0.2)

cdl_mosaic = Mosaic.from_product_bands(
    cdl_pid, "class", start_datetime="2021-01-01", end_datetime="2022-01-01"
)

## Interacting between ImageStacks and Mosaics
In the next few cells we will calculate NDVI through our time period and then mask to our Cropland Data Layer's Corn class:

In [ ]:
nir, red = s2_stack.unpack_bands("nir red")
ndvi = (nir - red) / (nir + red)

In [ ]:
ndvi_corn_mask = ndvi.mask(cdl_mosaic != 1)

In [ ]:
m = dc.map
m.center = 41.34232959809853, -95.54491138405865
m.zoom = 13

In [ ]:
(
    s2_stack.pick_bands("red green blue")
    .median(axis="images")
    .visualize("Sentinel-2 Composite", m)
)
ndvi.mean(axis="images").visualize("NDVI Composite", m)
cdl_mosaic.pick_bands("class").visualize("CDL", m, colormap="terrain")
ndvi_corn_mask.mean(axis="images").visualize(
    "NDVI Composite Corn Mask", m, colormap="magma"
)

In [ ]:
m

## Time Series Analysis and Aggregation with ImageStacks
In the following cells we will demonstrate how you can utilize an image stack to aggregate time series summary statistics and ndarrays.  There are two options for aggregation:

* __axis='pixels'__
    * First, we'll call .compute over our stack along __axis='pixels'__ to calculate the mean value _across each AOI for each image._ When calling __axis='pixels'__ we are _aggregating the spatial dimension to a single value_.
* __axis='images'__
    * Next we will explore calling .compute over our stack along the __axis='images'__ to build simple data composites. When computing over the images axis we will be aggregating all images in our stack into a single-image equivalent.

First we will define a tile over which we want to operate:

In [ ]:
dltile = dl.geo.DLTile.from_latlon(
    41.34232959809853, -95.54491138405865, tilesize=512, pad=0, resolution=10.0
)

Next retrieve the properties of our stack, such as __datetime__ and __image ID__, which we will keep track of for later:

In [ ]:
ndvi_props = ndvi_corn_mask.properties.compute(dltile)
ndvi_dates = [p["acquired"].strftime("%Y-%m-%d") for p in ndvi_props]
ndvi_ids = [p["id"] for p in ndvi_props]

Next we will explore __axis='pixels'__, where the returned value is a single statistic corresponding to an image's acquired date:

In [ ]:
ndvi_mean = ndvi.mean(axis="pixels").compute(dltile)
ndvi_mean_list = ndvi_mean.ndarray[:, 0].tolist()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ndvi_dates, ndvi_mean_list)

Comparing __axis='pixels'__ to __axis='images'__, we'll see the resulting ndarray is the same shape as that of a 3D mosaic, in this case each pixel represents the mean value throughout the time period:

In [ ]:
ndvi_mean_arr = ndvi_corn_mask.mean(axis="images").compute(dltile).ndarray
dl.utils.display(ndvi_mean_arr, colormap="viridis", figsize=(5, 5))

## Groupby
We can also call [`ImageStack.groupby()`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.groupby) and aggregate over useful temporal dimensions, as well as any arbitary attribute. 

Here we will group our stack by date within our time period:

In [ ]:
ndvi_corn_mask_groupby = ndvi_corn_mask.groupby(
    lambda x: date(x.acquired.year, x.acquired.month, x.acquired.day)
)
type(ndvi_corn_mask_groupby)

We then compute our grouped object:

In [ ]:
groupby_results = list(ndvi_corn_mask_groupby.groups.compute(dltile))
len(groupby_results)

And now we can iterate over each date present in our stack:

In [ ]:
fig, ax = plt.subplots(
    nrows=len(groupby_results), figsize=(10, 10 * len(groupby_results))
)

for i, (date, date_stack) in enumerate(groupby_results):
    ax[i].set_title(date.strftime("%Y-%m-%d"))
    ndarr = date_stack.median(axis="images").compute(dltile)
    ax[i].imshow(ndarr.ndarray[0])